In [ ]:
import numpy as np
import pandas as pd

num_parameters = 6
num_combinations = 100000#5000 

#dirichlet
alpha = np.ones(num_parameters)
#alpha = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5])
combinations = np.random.dirichlet(alpha, num_combinations)


columns = ['R1(HA)', 'R2(IA)', 'R3(NVP)', 'R4(AA)', 'R5(HEAA)', 'R6(IBOA)']
new_test_df = pd.DataFrame(columns=columns)


for combo in combinations:
    rounded = np.round(combo[:-1] / 0.01) * 0.01
    last_value = np.round((1 - np.sum(rounded)) / 0.01) * 0.01
    if 0 <= last_value <= 1:
        rounded = np.append(rounded, last_value)
        if not ((rounded[1] + rounded[2] > 0.7 and rounded[0] > 0.2) or (np.sum(rounded[[0, 2, 3]]) < 0.05 and rounded[4] > 0.05 and rounded[1] > 0.05)):
            new_test_df = new_test_df.append(pd.Series(rounded, index=columns), ignore_index=True)
        if len(new_test_df) == num_combinations:
            break

new_test_df['e_value'] = 0

new_test_df.to_csv('new_test_data2_PC8_1000000.csv', index=False)


In [ ]:
# Load the new test data
new_test_df = pd.read_csv('new_test_data2_PC8_1000000.csv')

# Normalize the monomer values using the previously fitted scaler
new_test_normalized = new_test_df[['R1(HA)', 'R2(IA)', 'R3(NVP)', 'R4(AA)', 'R5(HEAA)', 'R6(IBOA)', 'e_value']]
#new_test_normalized = pd.DataFrame(scaler.transform(new_test_normalized), columns=new_test_normalized.columns)

# Generate cross features
for monomer in ['R1(HA)', 'R2(IA)', 'R3(NVP)', 'R4(AA)', 'R5(HEAA)', 'R6(IBOA)']:
    for feature in monomers_info_df_normalized.columns:
        new_test_df[f'{monomer}_{feature}'] = new_test_df[monomer] * monomers_info_df.loc[monomer, feature]

# Select the relevant features for prediction
X_new_test = new_test_df[X_columns]
X_new_test = torch.FloatTensor(X_new_test.values)


In [ ]:
#new_test_df[X_columns]

In [ ]:
print(len(new_test_df[X_columns]))

In [ ]:
model.eval()
with torch.no_grad():
    y_new_pred = model(X_new_test)

# Convert predictions to NumPy array if needed
y_new_pred_np = y_new_pred.numpy()


# 使用PCA的逆变换进行数据重构
y_new_pred_reconstructed = pca.inverse_transform(y_new_pred_np)

# 确保它们是二维数组
#y_test_reconstructed = y_test_reconstructed.reshape(-1, 50, 2)
y_new_pred_reconstructed = y_new_pred_reconstructed.reshape(-1, 50, 2)
print("done with all predict")

# 手动创建 e_values 数组以确保包含整数0
e_values = np.arange(-1.5, 1.6, 0.3)  # 增加结束值以包括1.5
e_values = np.sort(np.unique(np.append(e_values, 0)))  # 确保包含0并且值唯一

In [ ]:
save_folder = 'Virtual Experiments_set4_toughness10up_yeild_4less_last_10less_strain_250up'
os.makedirs(save_folder, exist_ok=True)
for i in range(y_new_pred_reconstructed.shape[0]):

    strain = y_new_pred_reconstructed[i][:, 0]
    stress = y_new_pred_reconstructed[i][:, 1]

    max_strain = np.max(strain)
    last_stress=stress[-2]
    cut_off_index = int(len(strain) * 0.3)
    strain_30_percent = strain[:cut_off_index]
    stress_30_percent = stress[:cut_off_index]
 
    yield_strength = np.max(stress_30_percent)

    toughness = np.trapz(stress, strain)/100
    if yield_strength<4 and max_strain>250 and last_stress<10:

    #monomers_values = new_test_df.iloc[i][['R1(HA)', 'R2(IA)', 'R3(NVP)', 'R4(AA)', 'R5(HEAA)', 'R6(IBOA)']]
    #if monomers_values[0]==0.21 and monomers_values[1]==0.37 and monomers_values[2]==0.07 and monomers_values[3]==0.22:
        monomers_values = new_test_df.iloc[i][['R1(HA)', 'R2(IA)', 'R3(NVP)', 'R4(AA)', 'R5(HEAA)', 'R6(IBOA)']]
   

        filename = f'R1_{monomers_values["R1(HA)"]}_R2_{monomers_values["R2(IA)"]}_R3_{monomers_values["R3(NVP)"]}_R4_{monomers_values["R4(AA)"]}_R5_{monomers_values["R5(HEAA)"]}_R6_{monomers_values["R6(IBOA)"]}_laststress_{last_stress:.2f}_maxstrain_{max_strain:.2f}_toughness_{toughness:.2f}.png'

        print(new_test_df.iloc[i][['R1(HA)', 'R2(IA)', 'R3(NVP)', 'R4(AA)', 'R5(HEAA)', 'R6(IBOA)']])
        print("Last Stress (MPa):", last_stress)
        print("Max Strain (Breakage point):", max_strain)
        print("Toughness (Area under the curve)(MJ/m³):", toughness)

        plt.figure(figsize=(12, 6))
        plt.plot(strain, stress, alpha=0.5)
        plt.title('Stress-Strain Relationship')
        plt.xlabel('Strain (%)')
        plt.ylabel('Stress (MPa)')
        plt.xlim(left=0)
        plt.ylim(bottom=0)
        filename = filename.replace('.png', '_evalues.png')
        plt.savefig(os.path.join(save_folder, filename))
        plt.close()
        #plt.show()

print("done")